In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os, re
import numpy as np
import urllib.request as ur
import matplotlib.pyplot as plt
import folium
import base64

## 버거킹 매장 주소 습득

In [ ]:
stores_name = []
stores_address = []
stores_phonenum = []
url = 'https://www.burgerking.co.kr/'
chromedriver = './python_ex_100/selenium/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)

driver.get(url)
store_menu = driver.find_elements_by_css_selector("li.menu_open")[0]
store_menu.click() #매장소개 클릭
time.sleep(0.5)
store_find = store_menu.find_elements_by_css_selector("a")[0]
store_find.click() # 매장찾기 클릭
time.sleep(0.5)
# 아래부터 "https://www.burgerking.co.kr/#/store"임
address_select = driver.find_elements_by_css_selector("ul.item3")[0]
address_select = address_select.find_elements_by_css_selector("button")[2]
address_select.click() # 지역검색 클릭
time.sleep(0.5)
print('서울시내 매장 검색')
address_select = driver.find_elements_by_css_selector("div.form_list > select.st04")[0]
address_select = driver.find_elements_by_css_selector("option")[1]
address_select.click() # 서울특별시 클릭.

time.sleep(1)
store_object = driver.find_elements_by_css_selector("ul.list02 > li")
print('매장 정보 수집 끝')
for i, j in enumerate(store_object):
    stores_name.append(j.find_element_by_css_selector("div.shop_detail02 > p").text)
    
    temp=j.find_elements_by_css_selector("div.subinfo > p")
    
    if temp[0].text == '':
        stores_address.append('주소없음')
    else:
        stores_address.append(temp[0].text)
    if temp[1].text == '':
        stores_phonenum.append('번호없음')
    else:
        stores_phonenum.append(temp[1].text)

driver.quit()

store_dict = {"매장명":stores_name, "매장주소":stores_address, "전화번호":stores_phonenum}

    

create_ex = pd.DataFrame(store_dict)

create_ex.to_csv('./data/bugerking/bugerking.csv',encoding='utf-8-sig')
print('매장 정보 저장 끝')

## 롯데리아 매장정보 습득

In [ ]:
stores_name = []
stores_address = []
stores_phonenum = []
page_start=3
url = 'http://www.lotteria.com/Shop/Shop_List.asp'
chromedriver = './python_ex_100/selenium/chromedriver'
t = True
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)
# driver = webdriver.Chrome(chromedriver)

driver.get(url)
store_menu = driver.find_elements_by_css_selector("select#locationSelect > option")[1]

store_menu.click() #매장소개 클릭
time.sleep(0.5)
driver.find_element_by_css_selector("div.locationSelectWrap > a").click()
time.sleep(1.5)

end_page = True
while end_page:
    if stores_name != []:
        page_start = 2
    for i in driver.find_elements_by_css_selector("tr.shopSearch"):
        idx, stroe_code = tuple(re.sub("(goView\()*(\); return false;)*(\,)*(\')*","",i.find_element_by_css_selector("a").get_attribute('onclick')).split())
        response = requests.get("http://www.lotteria.com/Shop/Shop_View.asp?Idx="+idx+"&StoreCode="+stroe_code)
        soup = BeautifulSoup(response.text, "html.parser")
        
        stores_name.append(soup.select_one("div#content div.h3_box").text.strip())
        stores_address.append(soup.select_one("td.rt").text.strip())
        stores_phonenum.append(re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]))

    driver.find_elements_by_css_selector("div.paging_basic a")[2].click()
    
    time.sleep(2.5)
    for next_page in range(page_start,12):
        page_numbers = driver.find_elements_by_css_selector("div.paging_basic a")
        for i in driver.find_elements_by_css_selector("tr.shopSearch"):
            idx, stroe_code = tuple(re.sub("(goView\()*(\); return false;)*(\,)*(\')*","",i.find_element_by_css_selector("a").get_attribute('onclick')).split())
            response = requests.get("http://www.lotteria.com/Shop/Shop_View.asp?Idx="+idx+"&StoreCode="+stroe_code)
            soup = BeautifulSoup(response.text, "html.parser")
            if re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]) in stores_phonenum:
                end_page = False
                print('수집 끝')
                break
            stores_name.append(soup.select_one("div#content div.h3_box").text.strip())
            stores_address.append(soup.select_one("td.rt").text.strip())
            stores_phonenum.append(re.sub("( )*","",soup.select_one("td.phone").text.split("/")[0]))

        if end_page == False:
            break
        page_numbers[next_page].click()
        time.sleep(2.5)
    
store_dict = {"매장명":stores_name, "매장주소":stores_address, "전화번호":stores_phonenum}
create_ex = pd.DataFrame(store_dict)

create_ex.to_csv('./data/lotteria/lotteria.csv',encoding='utf-8-sig')
print('매장 정보 저장 끝')
driver.quit()

## csv에 저장된 점포 정보 이용해 네이버에서 검색가능한 id추출.
- 엑셀명 자동 수정기능 없음 알아서 기입해야함
- store_name: 변수의 매장명 아직 직접 수정해야함
- 정보 수집후 결측치 직접 수정 필요

In [ ]:
store_infos = pd.read_csv("./data/bugerking/bugerking.csv", index_col = 0) # 버거킹
store_name = '버거킹%20' # 버거킹

# store_infos = pd.read_csv("./data/lotteria/lotteria.csv", index_col = 0) # 롯데리아
# store_name = '롯데리아%20' # 롯데리아

url = 'view-source:https://pcmap.place.naver.com/restaurant/list?level=top&entry=pll&query='
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)
store_ids = []
for i in store_infos['매장명']:
    if i[-1] == '점':
        driver.get(url+store_name+i)
    else :
        driver.get(url+store_name+i+'점')
    time.sleep(5)
    result = driver.find_elements_by_css_selector("td.line-content")[29].text
    
    try :
        store_id = re.findall("(?<=\"id\":\"RestaurantSummary:).*(?=\",\"typename\":\"RestaurantSummary\")",result)
        if store_id == []:
            store_ids.append('id 없음')
        else :
            store_ids.append(store_id)
        print(store_id)
    except IndexError:
        store_ids.append('오류')
        
store_infos["id"] =  store_ids
store_infos.to_csv('./data/bugerking/bugerking_2.csv',encoding='utf-8-sig') # 버거킹
# store_infos.to_csv('./data/lotteria/lotteria_2.csv',encoding='utf-8-sig') # 롯데리아

## 네이버 검색을 위한 id 와 csv의 점포명 비교, 리뷰 갯수 추출

In [ ]:
store_infos = pd.read_csv("./data/bugerking/bugerking_2.csv", index_col = 0) # 버거킹
# store_infos = pd.read_csv("./data/lotteria/lotteria_2.csv", index_col = 0) # 롯데리아
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(chromedriver, chrome_options=options)
# driver = webdriver.Chrome(chromedriver)
review_counts=[]
for i, j in zip(store_infos['id'], store_infos['매장명']):
    url = 'https://m.place.naver.com/restaurant/'+i[2:-2]+'/review/visitor'
    driver.get(url)
    try :
        result = driver.find_element_by_css_selector("div#app-root h1._2msz7")
        result_2 = driver.find_element_by_css_selector("div.place_section div._1XXGh")
        review_counts.append(int(re.sub(",*","",result_2.text.split()[0][:-1])))
        print(j)
        print(result.text)
        print(int(re.sub(",*","",result_2.text.split()[0][:-1])))
    except :
        review_counts.append(0)
        
    
store_infos["review"] =  review_counts
store_infos.to_csv('./data/bugerking/bugerking_3.csv',encoding='utf-8-sig') # 버거킹
# store_infos.to_csv('./data/lotteria/lotteria_3.csv',encoding='utf-8-sig') # 롯데리아
driver.quit()

## 리뷰 갯수 기준 상위 50개의 점포정보 최종 저장
- 가정 : 네이버 리뷰가 많으면 요기요 리뷰도 많을 것이다.

In [ ]:
def str_process(n):
    return n[2:-2]
bugerking_data = pd.read_csv("./data/bugerking/bugerking_3.csv", index_col = 0)
lotteria_data = pd.read_csv("./data/lotteria/lotteria_3.csv", index_col = 0)
lotteria_data['id'] = lotteria_data['id'].apply(str_process) # id 형식 변경
bugerking_data['id'] = bugerking_data['id'].apply(str_process) # id 형식 변경
bugerking_data.sort_values('review', ascending = False)[:50].to_csv('./data/bugerking/bugerking_data.csv',encoding='utf-8-sig')
lotteria_data.sort_values('review', ascending = False)[:50].to_csv('./data/lotteria/lotteria_data.csv',encoding='utf-8-sig')

## 매장별 요기요 상세페이지 습득, 저장(csv)

In [ ]:
def yogiyo_store_url_find(address_list, name_list, brand_name, brand_type):
#     address_list = [i for i in pd.read_csv("./data/lotteria/lotteria_data.csv", index_col = 0)['매장주소']]
#     name_list = [i for i in pd.read_csv("./data/lotteria/lotteria_data.csv", index_col = 0)['매장명']]

    url = 'https://www.yogiyo.co.kr/mobile/?gclid=CjwKCAjw47eFBhA9EiwAy8kzNGbtPeXf1_78lj3sV4UkKJLnuA11hYXcCfZ9B62XZa8clZapkaMuixoCsaUQAvD_BwE#/'
    url_find = []

    re_try = False
    chromedriver = './python_ex_100/selenium/chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(chromedriver, chrome_options=options)
#     driver = webdriver.Chrome(chromedriver)
    for store_address, store_name in zip(address_list, name_list):
        driver.get(url)
        time.sleep(0.5)
        driver.find_elements_by_css_selector("div.input-group input")[0].click()
        time.sleep(0.5) # 검색 탭 클릭
        driver.find_elements_by_css_selector("div.input-group  button.btn-search-location-cancel.btn-search-location.btn.btn-default")[0].click()
        time.sleep(0.5) # 이미 채워져있는 텍스트 지우는 X버튼 클릭
        driver.find_elements_by_css_selector("div.input-group input")[0].send_keys(store_address)
        time.sleep(0.5) # 검색에 필요한 키워드 타이핑
        driver.find_elements_by_css_selector("div.input-group  button.btn.btn-default.ico-pick")[0].click()
        time.sleep(3) # 검색버튼 클릭
        search=driver.find_elements_by_css_selector("div.input-group ul.dropdown-menu.ng-scope.am-flip-x.bottom-left > li")
        # 검색 클릭시 하위리스트들 가져오기(검색 클릭시 바로 페이지가 넘어가는 경우도 있다.)
        if len(search) >= 3: # 리스트 길이가 3이상일때
            search[2].click() # 첫번째 주소 클릭시킴
        else :
            pass # 없다면 그냥 패스
        time.sleep(1)
        driver.find_elements_by_css_selector("div.list-option-inner option")[4].click()
        # 거리순 옵션 클릭

        time.sleep(1)
        restaurants = driver.find_elements_by_css_selector("div.ng-scope div.restaurant-list div.col-sm-6.contract")
        # 매장 정보들 전부 습득
        # 첫 로딩에 나온 매장만 가져온다.
        time.sleep(2)
        for restaurant in restaurants: # 가져온 매장리스트 기준으로 타깃(롯데리아) 주소 획득시작
            if brand_type == 0 and brand_name+'-{}'.format(store_name) in restaurant.text: #버거킹일때
                re_try = False
                restaurant.find_element_by_css_selector("div.col-sm-6.contract div.item.clearfix").click()
                print(store_name, driver.current_url)
                url_find.append([store_name, store_address, driver.current_url])
                break
            elif brand_type == 1 and brand_name in restaurant.text: # 롯데리아 일때
                re_try = False
                restaurant.find_element_by_css_selector("div.col-sm-6.contract div.item.clearfix").click()
                print(store_name, driver.current_url)
                url_find.append([store_name, store_address, driver.current_url])
                break
            else :
                re_try = True


        if re_try == True: # 찾지 못했을때 키워드를 변경하고 재검색한다.
            print("재검색")
            driver.find_elements_by_css_selector("div.input-group input")[0].click()
            time.sleep(0.5)
            driver.find_elements_by_css_selector("div.input-group  button.btn-search-location-cancel.btn-search-location.btn.btn-default")[0].click()
            time.sleep(0.5)
            driver.find_elements_by_css_selector("div.input-group input")[0].send_keys(store_address+' '+brand_name)
            time.sleep(0.5)
            driver.find_elements_by_css_selector("div.input-group  button.btn.btn-default.ico-pick")[0].click()
            time.sleep(2)
            re_search=driver.find_elements_by_css_selector("div.input-group ul.dropdown-menu.ng-scope.am-flip-x.bottom-left > li")
            if len(re_search) >= 3:
                re_search[2].click()
            else :
                pass
            time.sleep(2)
            driver.find_elements_by_css_selector("div.list-option-inner option")[4].click()
            time.sleep(0.5)
            restaurants = driver.find_elements_by_css_selector("div.ng-scope div.restaurant-list div.col-sm-6.contract")
            time.sleep(2)
            for restaurant in restaurants:
                if brand_type == 0 and brand_name+'-{}'.format(store_name) in restaurant.text: #버거킹일때
                    re_try = False
                    restaurant.find_element_by_css_selector("div.col-sm-6.contract div.item.clearfix").click()
                    print(store_name, driver.current_url)
                    url_find.append([store_name, store_address, driver.current_url])
                    break
                elif brand_type == 1 and brand_name in restaurant.text: # 롯데리아 일때
                    re_try = False
                    restaurant.find_element_by_css_selector("div.col-sm-6.contract div.item.clearfix").click()
                    print(store_name, driver.current_url)
                    url_find.append([store_name, store_address, driver.current_url])
                    break
                else :
                    pass
            if re_try == True:
                print("실패")
                url_find.append([store_name,'획득실패'])
        else :
            pass

    # 최종 url 정보는 url_find안에 담긴다.
    driver.quit()
    return url_find

In [ ]:
# address = [i for i in pd.read_csv("./data/lotteria/lotteria_data.csv", index_col = 0)['매장주소']]
# name = [i for i in pd.read_csv("./data/lotteria/lotteria_data.csv", index_col = 0)['매장명']]
# get_url_list = yogiyo_store_url_find(address, name, '롯데리아', 1)

address = [i for i in pd.read_csv("./data/bugerking/bugerking_data.csv", index_col = 0)['매장주소']]
name = [i for i in pd.read_csv("./data/bugerking/bugerking_data.csv", index_col = 0)['매장명']]
get_url_list = yogiyo_store_url_find(address, name, '버거킹', 0)

d_name = []
d_address = []
d_url = []

for i in get_url_list:
    if i[-1] == '획득실패':
        d_name.append(i[0])
        d_address.append('획득실패')
        d_url.append('획득실패')
    else :
        d_name.append(i[0])
        d_address.append(i[1])
        d_url.append(i[2])
    
dict_t = {}
dict_t['매장명'] = d_name
dict_t['주소'] = d_address
dict_t['url'] = d_url
dict_a = pd.DataFrame(dict_t)
# dict_a.to_csv('./data/lotteria/lotteria_urls.csv',encoding='utf-8-sig')
dict_a.to_csv('./data/bugerking/bugerking_urls.csv',encoding='utf-8-sig')

## 습득한 요기요 매장별 상세페이지 주소 이용 리뷰 크롤링

In [ ]:
def review_get(url_list):
    chromedriver = './python_ex_100/selenium/chromedriver'
    first_days = [] # 크롤링 종료일 설정
    store_dict = {}
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36")
    browser =  webdriver.Chrome(chromedriver, options = options)
    
    for i in range(1,31,1): # 마지막 날짜(크롤링 시작 날짜) 20년 5월
        first_days.append("2020년 5월 {}일".format(i))
    
    try:
        for url in url_list:
            
            browser.get(url)
            browser.maximize_window()
            time.sleep(1)
            selected_store_name = browser.find_element_by_css_selector("div.restaurant-title > span").text
            print(selected_store_name)
            
            Interval = 0.5
            time.sleep(Interval)

            review_button = browser.find_elements_by_css_selector('.nav.nav-tabs.restaurant-tab a')
            review_button[1].click()

            # 화면 가장 아래로 스크롤 내리기
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            # 현재 문서 높이를 가져와서 저장
            prev_height = browser.execute_script("return document.body.scrollHeight")

            while True:
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                try:
                    more = browser.find_element_by_css_selector('li.list-group-item.btn-more a')
                    more.click()
                except:
                    pass
                time.sleep(Interval)

                curr_height = browser.execute_script("return document.body.scrollHeight")
                if curr_height == prev_height:
                    break

                prev_height = curr_height

                # first_day에 정한 날짜까지 스크롤할 시 스크롤 중지
                if browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope span.review-time.ng-binding')[-1].text.strip() in first_days:
                    print("{}까지 스크롤 완료".format(browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope span.review-time.ng-binding')[-1].text.strip()))
                    break


            reviews = browser.find_elements_by_css_selector('li.list-group-item.star-point.ng-scope')

            # 맛, 양, 배달 별 별점, 리뷰 리스트 선언
            taste_ratings = []
            quantity_ratings = []
            delivery_ratings = []
            overall_reviews = []
            for i in reviews:
                ratings = i.find_element_by_css_selector('li.list-group-item.star-point.ng-scope span.category')
                taste_point = int(ratings.find_elements_by_css_selector('span')[-7].text)
                quantity_point = int(ratings.find_elements_by_css_selector('span')[-4].text)
                delivery_point = int(ratings.find_elements_by_css_selector('span')[-1].text)
                if taste_point <= 3 or quantity_point <= 3 or delivery_point <= 3:
                    taste_ratings.append(taste_point)
                    quantity_ratings.append(quantity_point)
                    delivery_ratings.append(delivery_point)
                    overall_reviews.append(i.find_element_by_css_selector('p').text)

            print('맛 평점: ', taste_ratings)
            print('양 평점: ', quantity_ratings)
            print('배달 평점: ', delivery_ratings)
            print('전체', overall_reviews)

            # 점포 별 별점리스트 및 리뷰 리스트 딕셔너리 선언
            print(store_dict)
            if store_dict == {}:
                store_dict = {'taste':taste_ratings, 'quantity':quantity_ratings, 'delivery':delivery_ratings, 'reviews':overall_reviews, 'store_name':selected_store_name}
                full_df = pd.DataFrame(store_dict)
            else :
                store_dict = {'taste':taste_ratings, 'quantity':quantity_ratings, 'delivery':delivery_ratings, 'reviews':overall_reviews, 'store_name':selected_store_name}
                temp_dict = pd.DataFrame(store_dict)
                full_df = pd.concat([full_df,temp_dict], ignore_index=True)
            print(full_df)
    except :
        print(selected_store_name)
        browser.quit()
        return full_df
    browser.quit()
    return full_df

In [ ]:
# file_name = 'lotteria/lotteria_urls'
# result_name = 'lotteria'

file_name = 'bugerking/bugerking_urls'
result_name = 'bugerking'

url_list = [i for i in pd.read_csv("./data/"+file_name+".csv", index_col = 0)['url']]

for n, i in enumerate(url_list):
    print(n+1)
    try:
        if n == 0:
            use_df = review_get2([i])
        else :
            use_df = pd.concat([use_df,review_get2([i])], ignore_index=True)
            print(use_df.head())
            print(use_df.tail())
    except:
        use_df.to_csv("./data/result/review_"+result_name+"_er.csv",encoding='utf-8-sig')
        # 오류 발생시 중간 저장후 종료.

use_df.to_csv("./data/result/review_"+result_name+".csv",encoding='utf-8-sig')

## 크롤링 후 빠진 매장 없는지 검사.

In [ ]:
use_df = pd.read_csv("./data/result/review_"+result_name+".csv",encoding='utf-8-sig')
print(len(use_df.groupby(use_df['store_name']).count()))

## 네이버 API(Geocode)이용 위도, 경도 습득

In [ ]:
def geocode_api(file_path):
    client_id = '본인이 할당받은 ID 입력'
    client_pw = '본인이 할당받은 Secret 입력'

    api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

    # 주소 목록 파일 (.csv)
    
    data = pd.read_csv(file_path, index_col = 0)
    # 주소 목록 파일 (.xlsx)

    # 네이버 지도 API 이용해서 위경도 찾기
    geo_coordi = []     
    for add in data['주소']:
        add_urlenc = parse.quote(add)  
        url = api_url + add_urlenc
        request = Request(url)
        request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
        request.add_header('X-NCP-APIGW-API-KEY', client_pw)
        try:
            response = urlopen(request)
        except HTTPError as e:
            print('HTTP Error!')
            latitude = None
            longitude = None
        else:
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read().decode('utf-8')
                response_body = json.loads(response_body)   # json
                if response_body['addresses'] == [] :
                    print("'result' not exist!")
                    latitude = None
                    longitude = None
                else:
                    latitude = response_body['addresses'][0]['y']
                    longitude = response_body['addresses'][0]['x']
                    print("Success!")
            else:
                print('Response error code : %d' % rescode)
                latitude = None
                longitude = None

        geo_coordi.append([latitude, longitude])


    np_geo_coordi = np.array(geo_coordi)
    pd_geo_coordi = pd.DataFrame({"매장명":data['매장명'].values, "주소": data['주소'].values,
                                  "위도": np_geo_coordi[:, 0],
                                  "경도": np_geo_coordi[:, 1]})
    return pd_geo_coordi

In [ ]:
# target = "./data/lotteria/lotteria"
target = "./data/bugerking/bugerking"
geo_df = geocode_api(target+"_urls.csv")
geo_df.to_csv(target+"_geo.csv",encoding='utf-8-sig')

## 지도에 첨부할 바차트 생성.
- import matplotlib.pyplot as plt
- 함수 정의후 이용(바차트 이미지 image폴더에 저장)
- "브랜드명"_geo2.csv 인 csv파일 생성(각 지점별 바차트 이미지 경로 포함)

In [ ]:
def make_bar_chart(file_path, file_name, brand):
    target = pd.read_csv(file_path+file_name+".csv", index_col = 0)
    x_name = ['맛', '양', '배송']

    for i in target.groupby(target['store_name']):
        plt.rc('font', family='Malgun Gothic')
        if brand == 0:
            plt.bar(x_name, [i[1].mean()[0], i[1].mean()[1], i[1].mean()[2]], color=['darkorange', 'sandybrown', 'moccasin'], linestyle='dotted')
            plt.title(i[0][4:])
        elif brand == 1:
            plt.bar(x_name, [i[1].mean()[0], i[1].mean()[1], i[1].mean()[2]], color=['red', 'maroon', 'firebrick'], linestyle='dotted')
            plt.title(i[0][5:-1])
        plt.ylim(1,5)
        plt.text(x_name[0], i[1].mean()[0],i[1].mean()[0].round(2), fontsize = 12, horizontalalignment='center', verticalalignment='bottom')
        plt.text(x_name[1], i[1].mean()[1],i[1].mean()[1].round(2), fontsize = 12, horizontalalignment='center', verticalalignment='bottom')
        plt.text(x_name[2], i[1].mean()[2],i[1].mean()[2].round(2), fontsize = 12, horizontalalignment='center', verticalalignment='bottom')

        if brand == 0:
            plt.savefig(file_path+"image/"+i[0][4:]+".png")
            plt.show()
        elif brand == 1:
            plt.savefig(file_path+"image/"+i[0][5:-1]+".png")
            plt.show()
    
    print("end")

In [ ]:
# 브랜드 별 바차트 생성
f_path = "./data/lotteria/"
f_name = "review_lotteria"
make_bar_chart(f_path, f_name, 1) # 롯데리아

f_path = "./data/bugerking/"
f_name = "review_bugerking"
make_bar_chart(f_path, f_name, 0) # 버거킹

In [2]:
# 엑셀에 점포별 워드크라우드 경로, 바차트 경로 추가.
def image_path_add(file_path, file_name):
    locations = pd.read_csv(f_path+f_name[7:]+"_geo.csv", index_col = 0)
    if 'bugerking' in f_name:
        locations['wd_cloud'] = locations['매장명'].apply(lambda x:f_path+"wordcloud/word_burgerking버거킹-"+x+'.png')
    elif 'lotteria' in f_name:
        locations['wd_cloud'] = locations['매장명'].apply(lambda x:f_path+"wordcloud/word_lotteria롯데리아-"+x+'점.png')
    locations['바_그래프'] = locations['매장명'].apply(lambda x:f_path+"image/"+x+'.png')
    locations.to_csv(f_path+f_name[7:]+"_geo2.csv", encoding='utf-8-sig')
    print("Create : "+f_path+f_name[7:]+"_geo2.csv")

In [ ]:
# 엑셀에 점포별 워드크라우드 경로, 바차트 경로 추가.
f_path = "./data/lotteria/"
f_name = "review_lotteria"
image_path_add(f_path, f_name) # 롯데리아

f_path = "./data/bugerking/"
f_name = "review_bugerking"
image_path_add(f_path, f_name)

## 지도에 마커 찍고 이미지 첨부
- !pip install folium
- import folium
- import base64

In [ ]:
def brand_map_create(file_path, file_name, brand):
    store_infos = pd.read_csv(file_path+file_name, index_col = 0) # 파일 오픈
    m = folium.Map(location=[37.503376,127.0475873],zoom_start=12) # 시작 위도 경도인것 같습다
    for i in store_infos.values:
        if brand == 0:
            store_name = i[0]
        elif brand == 1:
            store_name = i[0]+'점'
        pic1 = base64.b64encode(open(i[4],'rb').read()).decode() # 워드 클라우드 이미지 html에 올리기 위한 변환 과정입니다.
        pic2 = base64.b64encode(open(i[5],'rb').read()).decode() # 바차트 이미지 html에 올리기 위한 변환 과정입니다.
        image_tag = '<h1><p align="center">'+store_name+'</p></h1><br><img src="data:image/jpeg;base64,{0}"width="432", height="288"><br><br><img src="data:image/jpeg;base64,{1}">'.format(pic1, pic2) # 이미지 추가
        iframe = folium.IFrame(image_tag, width=450, height=750) # 마커 클릭시 창 크기 옵션인것 같습니다.
        popup = folium.Popup(iframe, max_width=600) # 팝업창 크기?
        folium.Marker([i[2], i[3]],popup=popup,tooltip=store_name).add_to(m) # 마커 추가입니다.
        

    m.save(file_path+file_name[:-9]+"_info.html") # 파일로 저정하는 부분입니다.


In [ ]:
bmc_f_path = "./data/lotteria/"
bmc_f_name = "lotteria_geo2.csv"
brand_map_create(bmc_f_path, bmc_f_name, 1) # 롯데리아

bmc_f_path = "./data/bugerking/"
bmc_f_name = "bugerking_geo2.csv"
brand_map_create(bmc_f_path, bmc_f_name, 0) # 버거킹